In [1]:
import numpy as np
import pandas as pd

In [2]:
#reading the data
df= pd.read_csv('dataset/application_train.csv')
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.shape

(307511, 122)

In [4]:
df.info

<bound method DataFrame.info of         SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0           100002       1         Cash loans           M            N   
1           100003       0         Cash loans           F            N   
2           100004       0    Revolving loans           M            Y   
3           100006       0         Cash loans           F            N   
4           100007       0         Cash loans           M            N   
...            ...     ...                ...         ...          ...   
307506      456251       0         Cash loans           M            N   
307507      456252       0         Cash loans           F            N   
307508      456253       0         Cash loans           F            N   
307509      456254       1         Cash loans           F            N   
307510      456255       0         Cash loans           F            N   

       FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
0        

In [5]:
df.CODE_GENDER.value_counts()

CODE_GENDER
F      202448
M      105059
XNA         4
Name: count, dtype: int64

In [6]:
print(df.columns.tolist())


['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELE

Preprocessing (to be implemented before visualization)

In [7]:
#optimize memory
def optimize_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Optimize numeric dtypes:
      - Integers → int8 / int16 / int32
      - Floats   → float16 / float32
    Returns a DataFrame with reduced memory usage.
    """
    before = df.memory_usage(deep=True).sum() / 1024**2
    optimized = df.copy()

    for col in optimized.columns:
        s = optimized[col]

        # --- Integers ---
        if pd.api.types.is_integer_dtype(s):
            vals = s.astype("int64")  # safe for bound checks
            if vals.min() >= np.iinfo(np.int8).min and vals.max() <= np.iinfo(np.int8).max:
                optimized[col] = s.astype("int8")
            elif vals.min() >= np.iinfo(np.int16).min and vals.max() <= np.iinfo(np.int16).max:
                optimized[col] = s.astype("int16")
            elif vals.min() >= np.iinfo(np.int32).min and vals.max() <= np.iinfo(np.int32).max:
                optimized[col] = s.astype("int32")
            # else: leave unchanged if too large

        # --- Floats ---
        elif pd.api.types.is_float_dtype(s):
            s64 = s.astype("float64")
            if np.allclose(s64, s64.astype("float16"), rtol=1e-03, atol=1e-06, equal_nan=True):
                optimized[col] = s64.astype("float16")
            else:
                optimized[col] = s64.astype("float32")

    after = optimized.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory: {before:.3f} MB → {after:.3f} MB ({(before - after) / before * 100:.2f}% reduction)")

    return optimized.shape


In [8]:
df.DAYS_BIRTH

0         -9461
1        -16765
2        -19046
3        -19005
4        -19932
          ...  
307506    -9327
307507   -20775
307508   -14966
307509   -11961
307510   -16856
Name: DAYS_BIRTH, Length: 307511, dtype: int64

In [9]:
#1.Convert ages: AGE_YEARS = -DAYS_BIRTH / 365.25
df['AGE_YEARS'] = (-df['DAYS_BIRTH']) / 365.25
df.AGE_YEARS

0         25.902806
1         45.900068
2         52.145106
3         52.032854
4         54.570842
            ...    
307506    25.535934
307507    56.878850
307508    40.974675
307509    32.747433
307510    46.149213
Name: AGE_YEARS, Length: 307511, dtype: float64

In [10]:
df.DAYS_EMPLOYED

0           -637
1          -1188
2           -225
3          -3039
4          -3038
           ...  
307506      -236
307507    365243
307508     -7921
307509     -4786
307510     -1262
Name: DAYS_EMPLOYED, Length: 307511, dtype: int64

In [11]:
#2.Employment tenure: EMPLOYMENT_YEARS = -DAYS_EMPLOYED / 365.25 (clip huge positives used as “not employed” codes)
   #special placeholder value 365243--It means the person is not currently employed.
df["DAYS_EMPLOYED"] = df["DAYS_EMPLOYED"].replace(365243, np.nan)
df["EMPLOYMENT_YEARS"] = -df["DAYS_EMPLOYED"] / 365.25
df["EMPLOYMENT_YEARS"] = df["EMPLOYMENT_YEARS"].clip(lower=0, upper=60)

In [12]:
# 3.Create ratios:
# DTI = AMT_ANNUITY / AMT_INCOME_TOTAL
# LOAN_TO_INCOME = AMT_CREDIT / AMT_INCOME_TOTAL
# ANNUITY_TO_CREDIT = AMT_ANNUITY / AMT_CREDIT
df["DTI"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]
df["LTI"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]
df["ANNUITY_TO_CREDIT"] = df["AMT_ANNUITY"] / df["AMT_CREDIT"]

In [13]:
df.shape

(307511, 127)

In [14]:
# 4.Handle missing values (report % and apply strategy: drop columns > 60% missing; impute median/most-frequent for others)
missing_percent = df.isnull().mean() * 100
missing_percent

SK_ID_CURR             0.000000
TARGET                 0.000000
NAME_CONTRACT_TYPE     0.000000
CODE_GENDER            0.000000
FLAG_OWN_CAR           0.000000
                        ...    
AGE_YEARS              0.000000
EMPLOYMENT_YEARS      18.007161
DTI                    0.003902
LTI                    0.000000
ANNUITY_TO_CREDIT      0.003902
Length: 127, dtype: float64

In [15]:
drop_cols = missing_percent[missing_percent > 60].index.tolist()
df.drop(columns=drop_cols, inplace=True)


In [16]:
for col in df.columns:
    if df[col].dtype in ["int64", "float64"]:
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

In [17]:
#5.Standardize categories (merge rare categories under “Other” if share < 1%)
def merge_rare(series, threshold=0.01):
    freqs = series.value_counts(normalize=True)
    rare = freqs[freqs < threshold].index
    return series.replace(rare, "Other")

cat_cols = df.select_dtypes(include=["object"]).columns
for col in cat_cols:
    df[col] = merge_rare(df[col])

    #basically goes through categorical coulmns and checks the frequecy of each category and if the frequency is less than 1% it replaces it with "Other"

In [18]:
#6.Outlier handling (winsorize top/bottom 1% for numerics)
# -------------------------
def winsorize(series):
    lower, upper = series.quantile([0.01, 0.99])
    return np.clip(series, lower, upper)

num_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[num_cols] = df[num_cols].apply(winsorize)

# It calculates the lower bound = 1st percentile (Q0.01)
# It calculates the upper bound = 99th percentile (Q0.99)
# Then:
# If a value < lower → replace it with the 1st percentile value
# If a value > upper → replace it with the 99th percentile value
# Otherwise → keep it as it is

In [19]:
#7.Define income brackets (quantiles: Low-Q1, Mid-Q2/Q3, High-Q4)
df["INCOME_BRACKET"] = pd.qcut(
    df["AMT_INCOME_TOTAL"],
    q=[0, 0.25, 0.75, 1.0],
    labels=["Low", "Mid", "High"]
)
#binning of income into categories (Low, Mid, High) using pd.qcut.

C:\Users\harsh\AppData\Local\Temp\ipykernel_20212\3561536839.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["INCOME_BRACKET"] = pd.qcut(


In [20]:
#8.Ensure consistent filters
# -------------------------
df["CODE_GENDER"] = df["CODE_GENDER"].replace({"XNA": "Other"})
df["NAME_EDUCATION_TYPE"] = df["NAME_EDUCATION_TYPE"].replace({"Academic degree": "Higher education"})
df["NAME_FAMILY_STATUS"] = df["NAME_FAMILY_STATUS"].replace({"Unknown": "Other"})
df["NAME_HOUSING_TYPE"] = df["NAME_HOUSING_TYPE"].replace({"Co-op apartment": "Other"})
#maps certain rare/unwanted categories to broader categories to simplify categorical variables.
print("✅ Preprocessing complete")
print("Final shape:", df.shape)

✅ Preprocessing complete
Final shape: (307511, 111)


In [21]:
#Outliers
def find_outliers_iqr(df: pd.DataFrame, cols=None) -> dict:
    """
    Detect outliers in numeric columns using IQR method.
    Returns a dictionary of column -> list of outlier indices.
    """
    if cols is None:
        cols = df.select_dtypes(include=np.number).columns

    outliers = {}
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        mask = (df[col] < lower) | (df[col] > upper)
        outliers[col] = df[mask].index.tolist()
    return outliers

In [22]:
df.CODE_GENDER.value_counts()

CODE_GENDER
F        202448
M        105059
Other         4
Name: count, dtype: int64

In [23]:
# Save cleaned dataset
df.to_csv("dataset/application_train.csv", index=False)


In [24]:
optimize_dataframe(df)

c:\Users\harsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\harsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\harsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\harsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
c:\Users\harsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


Memory: 479.382 MB → 304.302 MB (36.52% reduction)


(307511, 111)

In [25]:
df_sampled = df.sample(n=20000, random_state=2)

# Save the reduced clean dataset
df_sampled.to_csv("reduced_dataset.csv", index=False)